<a href="https://colab.research.google.com/github/aryaar04/SECURITY-bank-app/blob/main/bank_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

secure bank app backend using FastAPI, bcrypt, and JWT authentication, built with security best practices

 Features with Security Covered:
Password hashing with bcrypt (via passlib)

JWT token-based login

Input validation with pydantic

Defense against rainbow tables (via salted hashing)

No plain-text passwords

Token expiration and scope

HTTPS endpoint via ngrok (for secure API testing)

In [2]:
# 1. Install required packages, including the missing email-validator
!pip install fastapi uvicorn nest-asyncio pyngrok python-jose[cryptography] passlib[bcrypt] email-validator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.9 MB/s eta 0:00:00


In [5]:
pip install python-multipart

In [6]:
import nest_asyncio
from fastapi import FastAPI, HTTPException, Depends, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel, EmailStr
from passlib.context import CryptContext
from jose import JWTError, jwt
from datetime import datetime, timedelta
from typing import Optional
from pyngrok import ngrok
import uvicorn

# Allow multiple async loops in Colab
nest_asyncio.apply()

app = FastAPI()

# In-memory user store (not persistent)
fake_users_db = {}

# Security settings
SECRET_KEY = "a-very-strong-secret-key-for-jwt"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Pydantic models
class UserIn(BaseModel):
    email: EmailStr
    password: str

class UserOut(BaseModel):
    email: EmailStr
    balance: float

class Token(BaseModel):
    access_token: str
    token_type: str

# Utility functions
def get_password_hash(password):
    return pwd_context.hash(password)

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def create_access_token(data: dict, expires_delta: Optional[timedelta] = None):
    to_encode = data.copy()
    expire = datetime.utcnow() + (expires_delta or timedelta(minutes=15))
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def get_current_user(token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        email: str = payload.get("sub")
        if email is None or email not in fake_users_db:
            raise credentials_exception
        return fake_users_db[email]
    except JWTError:
        raise credentials_exception

# API endpoints
@app.post("/register", response_model=UserOut)
def register(user: UserIn):
    if user.email in fake_users_db:
        raise HTTPException(status_code=400, detail="Email already registered")
    hashed_pw = get_password_hash(user.password)
    fake_users_db[user.email] = {
        "email": user.email,
        "hashed_password": hashed_pw,
        "balance": 0.0
    }
    return UserOut(email=user.email, balance=0.0)

@app.post("/token", response_model=Token)
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    user = fake_users_db.get(form_data.username)
    if not user or not verify_password(form_data.password, user["hashed_password"]):
        raise HTTPException(status_code=401, detail="Invalid credentials")
    token = create_access_token(data={"sub": user["email"]}, expires_delta=timedelta(minutes=30))
    return {"access_token": token, "token_type": "bearer"}

@app.get("/account", response_model=UserOut)
def read_account(current_user: dict = Depends(get_current_user)):
    return UserOut(email=current_user["email"], balance=current_user["balance"])

@app.post("/deposit")
def deposit(amount: float, current_user: dict = Depends(get_current_user)):
    if amount <= 0:
        raise HTTPException(status_code=400, detail="Amount must be positive.")
    current_user["balance"] += amount
    return {"message": "Deposit successful", "balance": current_user["balance"]}


In [24]:
# 1. Install dependencies (run this only once)
!pip install fastapi uvicorn nest_asyncio pyngrok passlib[bcrypt] python-jose

# 2. Minimal app setup
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pyngrok import ngrok, conf
from threading import Thread
import time

# 3. Allow nested loops in Jupyter
nest_asyncio.apply()

# 4. Create FastAPI app
app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "🎉 FastAPI is running with ngrok!"}

# 5. Start server + expose via ngrok
def start():
    # Close previous tunnels
    ngrok.kill()

    # Replace with your actual token
    conf.get_default().auth_token = "2wPEN6vzJ5FYBGfqoN4vdAGn35t_6q3wYt5UpdBix74euAvZr"

    tunnel = ngrok.connect(8000)
    public_url = tunnel.public_url
    print("✅ Public URL:", public_url + "/docs")

    uvicorn.run(app, host="0.0.0.0", port=8000)

# 6. Start in a new thread
Thread(target=start).start()
time.sleep(5)  # Wait to ensure ngrok is ready


✅ Public URL: https://3cae-35-188-235-137.ngrok-free.app/docs


INFO:     Started server process [740]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
